# Getting Started with invertmeeg

This notebook walks through the core workflow:
1. Create a forward model and EEG montage
2. Simulate EEG data
3. Recover sources with the OmniChampagne solver
4. Visualize results

In [ ]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import mne
import numpy as np

from invert import Solver
from invert.forward import create_forward_model, get_info
from invert.simulate import SimulationConfig, SimulationGenerator

## 1. Create a Forward Model

We use the fsaverage template with a standard EEG montage. `ico2` gives a
small source space that is fast to compute.

In [ ]:
info = get_info(kind="biosemi32")
fwd = create_forward_model(sampling="ico3", info=info)

leadfield = fwd["sol"]["data"]
print(f"Leadfield shape: {leadfield.shape}  (channels x dipoles)")

## 2. Simulate EEG Data

We generate one sample with a few focal sources and moderate noise.

In [ ]:
config = SimulationConfig(
    batch_size=1,
    n_sources=(2, 2),
    n_orders=(0, 3),
    snr_range=(5, 5),
    n_timepoints=50,
)
gen = SimulationGenerator(fwd, config=config)
x, y, sim_info = next(gen.generate())

print(f"EEG data shape:    {x.shape}  (batch, channels, time)")
print(f"Source data shape:  {y.shape}  (batch, dipoles, time)")

### Wrap the simulated data in an `mne.EvokedArray`

The solver expects an MNE data object, so we create one from our numpy array.

In [ ]:
evoked = mne.EvokedArray(x[0], info, tmin=0)
evoked.plot(time_unit="s")
plt.show()

## 3. Compute the Inverse Solution with OmniChampagne

In [ ]:
solver = Solver("laura")
# solver = Solver("esmv")
# solver = Solver("apse")
solver.make_inverse_operator(fwd, evoked)
stc = solver.apply_inverse_operator(evoked)

print(f"SourceEstimate shape: {stc.data.shape}  (dipoles x time)")

## 4. Visualize the Results

### 4b. Ground truth SourceEstimate on the brain

In [ ]:
import os

fs_dir = mne.datasets.fetch_fsaverage(verbose=0)
subjects_dir = os.path.dirname(fs_dir)

src = fwd["src"]
vertices = [src[0]["vertno"], src[1]["vertno"]]
sfreq = info["sfreq"]

stc_true = mne.SourceEstimate(
    y[0], vertices=vertices, tmin=0, tstep=1.0 / sfreq, subject="fsaverage"
)

### 4c. Plot reconstruction on the cortical surface

Pick the time point with maximum activation and plot both ground truth and
reconstruction side by side.

In [ ]:
peak_time = stc.times[np.argmax(np.max(np.abs(stc.data), axis=0))]

stc.subject = "fsaverage"
plot_kwargs = dict(
    subjects_dir=subjects_dir,
    subject="fsaverage",
    surface="inflated",
    hemi="both",
    cortex="low_contrast",
    initial_time=peak_time,
    time_unit="s",
    size=(800, 400),
    smoothing_steps=10,
    verbose=0,
)

brain_true = stc_true.plot(brain_kwargs=dict(title="Ground Truth"), **plot_kwargs)
brain_est = stc.plot(brain_kwargs=dict(title=solver.name), **plot_kwargs)